# Crear un modelo de machine learning para predecir probabilidades de cancelación y crear clústers de usuarios
**Problema**: Pérdida de clientes -> calcular la pérdida. <span style="color: gold;">**Se define como pérdida cuando no viene en 1 mes**</span>.

**Tarea**: Elaborar una estrategia de retención de clientes.

**Pasos a seguir**
1) Predecir la probabilidad de pérdida
2) Elaborar y analizar clústers de usuarios 
3) Analizar factores que más impactan en la pérdida de clientes
4) Realizar recomendaciones
    * identificar a los grupos objetivo;
    * sugerir medidas para reducir la rotación;
    * describir cualquier otro patrón que observes con respecto a la interacción con los clientes.

## Diccionario de datos
* ``'Churn'`` — la cancelación para el mes en cuestión
* ``'gender'`` — género
* ``'Near_Location'`` — si el/la usuario/a vive o trabaja en el vecindario donde se encuentra el gimnasio.
* ``'Partner'`` — si el/la usuario/a trabaja en una compañía asociada (el gimnasio tiene empresas asociadas cuyos empleados obtienen descuentos; en esos casos el gimnasio almacena información sobre los empleadores de los clientes).
* ``'Promo_friends'`` — si el/la usuario/a originalmente se inscribió mediante una oferta “trae a un/a amigo/a” (se utilizó el código promocional de un/a amigo/a cuando pagaron el primer abono).
* ``'Phone'`` — si el/la usuario/a aportó el número de teléfono.
* ``'Age'`` — edad.
* ``'Lifetime'`` — el tiempo (en meses) desde que el/la usuario/a llegó por primera vez al gimnasio.
* ``'Contract_period'`` — 1 mes, 3 meses, 6 meses o 1 año.
* ``'Month_to_end_contract'`` — los meses que faltan hasta que expire el contrato.
* ``'Group_visits'`` — si el/la usuario/a participa en sesiones grupales.
* ``'Avg_class_frequency_total'`` — frecuencia media de visitas por semana a lo largo de la vida del cliente.
* ``'Avg_class_frequency_current_month'`` — frecuencia media de visitas por semana durante el mes en curso.
* ``'Avg_additional_charges_total'`` — cantidad total de dinero gastado en otros servicios del gimnasio: cafetería, productos deportivos, cosméticos, masajes, etc.